# <span style='color:orange'> Modelling Chinese Word Segmentation as Sequence to Sequence Prediction Problem </span>

# Generate data for sequence to sequence modelling

This returns the structure "data" which contains sentences as individual lists, along with the class label of each character. 

In [1]:
import numpy as np
import collections
from sklearn.model_selection import train_test_split
import keras
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
# we open the count file, get the word and assign labels to each character
# cannot use dictionaries, since the same character may appear again and overwrites the value at its place in dict.
def generateTupleListAccToSentences(filename='/local-scratch/asa224/wseg_simplified_cn.txt'):
    """
    This function generates a data list of lists, which contains sequences and corresponding
    labels for each character, according to the sentences in the input file. This function 
    takes the whole training set txt file as input, and generates sequences according to the 
    line, ie. each sequence is a line. 
    
    INPUT: Input to this function is the training text file
    
    If you want to use this data as training data for LSTM, you have to pad the sequences 
    since they are not of the same length. 
    """
    # filepath = '/mnt/D82A1A8F2A1A6B30/wseg_simplified_cn.txt'
    with(open(filepath, 'rb')) as f:
        data = [[]]
        count = 0
        for line in f:
            line = unicode(line, 'utf-8')
            line = line.replace('\n', '')
            words = line.split(' ')

            for word in words:
                if len(word) == 1:
                    data[count].append((word[0], 3))
                else:
                    for i, character in enumerate(word):
                        if i == 0: # this is the first letter
                            data[count].append((character, 0))
                        elif i == (len(word) - 1): # this is the last letter
                            data[count].append((character, 2))
                        else: # this is somewhere in the middle
                            data[count].append((character, 1))
            data.append([])
            count += 1

        f.close()
        
        return data
    
def generateWordFile(filename='/local-scratch/asa224/wseg_simplified_cn.txt'):
    """
    The function generates the word file, similar to the count_1w.txt file provided by Prof. Anoop
    
    The output of the file can be used to parse the characters, and is an input to the 
    generateTupleList() function as well. 
    
    INPUT: Input to this function is the training text file. 
    
    """
    with(open(filename, 'rb')) as f:
        word_file_1M = open('/local-scratch/word_file_1M', 'wb')
        for line in f:
            line = unicode(line, 'utf-8')
            line = line.replace('\n', '')
            line = line.split(' ')
            for word in line:
                word_file_1M.write(word.encode('utf-8') + '\t'.encode('utf-8') + str(0).encode('utf-8') +\
                                           '\n'.encode('utf-8'))
        f.close()
    word_file_1M.close()

def generateInputWordFile(filename='/local-scratch/asa224/input'):
    """
    The function generates the word file, similar to the count_1w.txt file provided by Prof. Anoop
    
    THIS GENERATES WORD FILE FOR INPUT TEXT. 
    
    The output of the file can be used to parse the characters, and is an input to the 
    generateTupleList() function as well. 
    
    INPUT: Input to this function is the training text file. 
    
    """
    with(open(filename, 'rb')) as f:
        word_file_1M = open('/local-scratch/input_word_file', 'wb')
        for line in f:
            line = unicode(line, 'utf-8')
            line = line.replace('\n', '')
            for word in line:
                word_file_1M.write(word.encode('utf-8') + '\t'.encode('utf-8') + str(0).encode('utf-8') +\
                                           '\n'.encode('utf-8'))
        f.close()
    word_file_1M.close()

def generateTupleList(filename='/local-scratch/asa224/word_file_1M'):
    """
    This function is similar to the above function in the sense that it assigns labels to each
    character in the training set. The function returns a list of tuples, in which each tuple
    contains a single character and its corresponding label. 
    
    INPUT: Input to this function is a WORD FILE generated by generateWordFile(filename) function. 
    
    Use this function in conjunction with nGramSequenceGenerator(labelledlist, n) to create a
    training set with constant sequence size, which does not require paddings. 
    """
    with(open(filename, 'rb')) as f:
        label = []
        for line in f:
            word, count = line.split('\t')

            # making sure the parsing is going fine
            assert int(count) == 0

            word = unicode(word, 'utf-8')
            if len(word) == 1:
                label.append((word[0], 3))
            else:
                for i, character in enumerate(word):
                    if i == 0: # this is the first letter
                        label.append((character, 0))
                    elif i == (len(word) - 1): # this is the last letter
                        label.append((character, 2))
                    else: # this is somewhere in the middle
                        label.append((character, 1))

        f.close()
        return label
    
def nGramSequenceGenerator(labelledlist, n):
    """
    Takes as input the label list of tuples generated by the code above. 
    The function generates sequence of size "n" from the given list. 
    """
    count = len(labelledlist)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( labelledlist[i*n : (i+1)*n])
    return ngrammedlist

In [3]:
list_of_tuples = generateTupleList(filename='/local-scratch/asa224/word_file_1M')
data = nGramSequenceGenerator(list_of_tuples, n=10)

In [4]:
data[0:5]

[[(u'\u7531', 0),
  (u'\u4e8e', 2),
  (u'\u5927', 0),
  (u'\u9646', 2),
  (u'\u8d74', 3),
  (u'\u53f0', 3),
  (u'\u65c5', 0),
  (u'\u6e38', 2),
  (u'\u8003', 0),
  (u'\u5bdf', 2)],
 [(u'\u56e2', 3),
  (u'\u7684', 3),
  (u'\u884c', 0),
  (u'\u7a0b', 1),
  (u'\u5b89', 1),
  (u'\u6392', 2),
  (u'\u5341', 0),
  (u'\u5206', 2),
  (u'\u7d27', 0),
  (u'\u51d1', 2)],
 [(u'\uff0c', 3),
  (u'\u56e2', 0),
  (u'\u5458', 1),
  (u'\u4eec', 2),
  (u'\u66f4', 0),
  (u'\u591a', 1),
  (u'\u7684', 2),
  (u'\u65f6', 0),
  (u'\u5019', 2),
  (u'\u53ea', 3)],
 [(u'\u80fd', 3),
  (u'\u662f', 3),
  (u'\u5728', 3),
  (u'\u65c5', 0),
  (u'\u884c', 2),
  (u'\u8f66', 3),
  (u'\u4e0a', 3),
  (u'\u8d70', 0),
  (u'\u9a6c', 1),
  (u'\u89c2', 1)],
 [(u'\u82b1', 1),
  (u'\u5730', 2),
  (u'\u8003', 0),
  (u'\u5bdf', 2),
  (u'\u53f0', 0),
  (u'\u5317', 2),
  (u'\u5e02', 3),
  (u'\u5bb9', 3),
  (u'\uff0c', 3),
  (u'\u6240', 0)]]

# Build initial integer embeddings

In [5]:
all_chars = [data[i][j][0] for i in range(0, len(data)) for j in range(0, len(data[i]))]

In [6]:
len(all_chars)

41617380

In [7]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [8]:
orig_dict, ret_dict = build_dataset(all_chars)
del all_chars

In [9]:
ret_dict[orig_dict[u'\u6743']]

u'\u6743'

In [10]:
len(orig_dict)

5419

Save the lookup dictionaries

In [30]:
import cPickle as pickle
pickle.dump( orig_dict, open( "orig_dict.p", "wb" ) )
pickle.dump( ret_dict, open( "ret_dict.p", "wb" ) )

## Create sequences suitable for training

In [11]:
x = [[]]
y = [[]]
for i in range(0, len(data)): # iterate over the whole dataset
    for j in range(0, len(data[i])): # iterate over the current sentence
        x[i].append(orig_dict[data[i][j][0]])
        y[i].append(data[i][j][1])
    x.append([])
    y.append([])

In [12]:
del data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=False, random_state=42)

In [ ]:
del x,y

In [14]:
print('Size of the Dataset\n')
print('X_TRAIN: {}'.format(np.shape(X_train)))
print('Y_TRAIN: {}'.format(np.shape(y_train)))

print('X_TEST: {}'.format(np.shape(X_test)))
print('Y_TEST: {}'.format(np.shape(y_test)))

Size of the Dataset

X_TRAIN: (3329391, 10)
Y_TRAIN: (3329391, 10)
X_TEST: (832348,)
Y_TEST: (832348,)


## Convert the labels to categorical. 

This is what they look like now

In [15]:
for i in y_train:
    print(i)
    break

[0, 2, 0, 2, 3, 3, 0, 2, 0, 2]


### Start conversion

Testing labels <br>
There are empty lists in the test set, let's remove them. 

In [16]:
X_train = [t for t in X_train if t != []]
y_train = [t for t in y_train if t != []]

X_test = [t for t in X_test if t != []]
y_test = [t for t in y_test if t != []]

Training labels

In [17]:
for i in range(0, len(y_train)):
    y_train[i] = to_categorical(y_train[i], num_classes=4)

In [18]:
for i in range(0, len(y_test)):
    y_test[i] = to_categorical(y_test[i], num_classes=4)

This how it looks now

In [19]:
for i in y_train:
    print(i)
    break

[[ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]]


Firstly, we must update the get_sequence() function to reshape the input and output sequences to be 3-dimensional to meet the expectations of the LSTM. The expected structure has the dimensions [samples, timesteps, features]. The classification problem has 1 sample (e.g. one sequence), a configurable number of timesteps, and one feature per timestep.

Let's reshape the training and testing data

### No need to pad sequences when working on constant size sequences

In [20]:
# # maximum length of the sequence
# maxval = len(max(X_train,key=len))

In [21]:
# X_train = sequence.pad_sequences(X_train, value=-1, maxlen=100)
# y_train = sequence.pad_sequences(y_train, value=-1, maxlen=100)

# X_test = sequence.pad_sequences(X_test, value=-1, maxlen=100)
# y_test = sequence.pad_sequences(y_test, value=-1, maxlen=100)

In [22]:
embedding = True

In [23]:
# reshape input and output data to be suitable for LSTMs
if embedding == False:
    n_timesteps = 10
    X_train = np.array(X_train).reshape(np.shape(X_train)[0], n_timesteps, 1)
    y_train = np.array(y_train).reshape(np.shape(y_train)[0], n_timesteps, 4)

    X_test = np.array(X_test).reshape(np.shape(X_test)[0], n_timesteps, 1)
    y_test = np.array(y_test).reshape(np.shape(y_test)[0], n_timesteps, 4)
    
else:
    # reshape input and output data to be suitable for LSTMs
    n_timesteps = 10
    X_train = np.array(X_train).reshape(np.shape(X_train)[0], n_timesteps)
    y_train = np.array(y_train).reshape(np.shape(y_train)[0], n_timesteps, 4)

    X_test = np.array(X_test).reshape(np.shape(X_test)[0], n_timesteps)
    y_test = np.array(y_test).reshape(np.shape(y_test)[0], n_timesteps, 4)

## IF you use embedding layer, use this code

In [24]:
X_train.shape

(3329391, 10)

TODO: Add embedding layer <br>
TODO: Increase LSTM units <br>
TODO: Decrease the sequence size <br>
TODO: Decrease the sequence size from the training set generation itself <br>

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Masking, Embedding, Dropout
# define LSTM
model = Sequential()
model.add(Embedding(input_dim=len(orig_dict), output_dim=200, input_length=10)) # dictionary size, embedding vector size, timesteps
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(200, return_sequences=True))
model.add(Dropout(0.3))
model.add(TimeDistributed(Dense(4, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 200)           1083800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 200)           320800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 200)           320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 200)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 4)             804       
Total params: 1,726,204
Trainable params: 1,726,204
Non-trainable params: 0
_________________________________________________________________


In [27]:
history = model.fit(X_train, y_train, epochs=1, 
          batch_size=100, verbose=1, validation_data=[X_test, y_test])
model.save('/home/asa224/Desktop/students_less_asa224/Test Folder on Less/model_epoch1.h5')

Train on 3329391 samples, validate on 832347 samples
Epoch 1/1
3329391/3329391 [==============================] - 1132s - loss: 0.4237 - acc: 0.8340 - val_loss: 0.3445 - val_acc: 0.8652


In [28]:
history2 = model.fit(X_train, y_train, epochs=1, 
          batch_size=100, verbose=1, validation_data=[X_test, y_test])
model.save('/home/asa224/Desktop/students_less_asa224/Test Folder on Less/model_epoch2.h5')

Train on 3329391 samples, validate on 832347 samples
Epoch 1/1
3329391/3329391 [==============================] - 1106s - loss: 0.3291 - acc: 0.8732 - val_loss: 0.3109 - val_acc: 0.8790


In [29]:
history3 = model.fit(X_train, y_train, epochs=1, 
          batch_size=100, verbose=1, validation_data=[X_test, y_test])
model.save('/home/asa224/Desktop/students_less_asa224/Test Folder on Less/model_epoch3.h5')

Train on 3329391 samples, validate on 832347 samples
Epoch 1/1
3329391/3329391 [==============================] - 1083s - loss: 0.3054 - acc: 0.8819 - val_loss: 0.2948 - val_acc: 0.8850


# Testing the model on input data